# mlpackage

## Fix seed value

In [1]:
import random

import torch


def set_seed(manualSeed: int):
    # Fix the seed value
    random.seed(manualSeed)
    torch.manual_seed(manualSeed)
    torch.cuda.manual_seed_all(manualSeed)
    # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = (
        True  # if True, causes cuDNN to only use deterministic convolution algorithms.
    )
    torch.backends.cudnn.benchmark = False  # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.


class WorkerInitializer:
    def __init__(self, manualSeed: int):
        self.manualSeed = manualSeed

    def worker_init_fn(self, worker_id: int):
        random.seed(self.manualSeed + worker_id)

In [2]:
manualSeed = 42
set_seed(manualSeed)

## Create ResNet20

In [3]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init


# ResNet for CIFAR
class DownsampleA(nn.Module):
    def __init__(self, nIn, nOut, stride):
        super(DownsampleA, self).__init__()
        assert stride == 2
        self.avg = nn.AvgPool2d(kernel_size=1, stride=stride)

    def forward(self, x):
        x = self.avg(x)
        return torch.cat((x, x.mul(0)), 1)


class DownsampleC(nn.Module):
    def __init__(self, nIn, nOut, stride):
        super(DownsampleC, self).__init__()
        assert stride != 1 or nIn != nOut
        self.conv = nn.Conv2d(
            nIn, nOut, kernel_size=1, stride=stride, padding=0, bias=False
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class DownsampleD(nn.Module):
    def __init__(self, nIn, nOut, stride):
        super(DownsampleD, self).__init__()
        assert stride == 2
        self.conv = nn.Conv2d(
            nIn, nOut, kernel_size=2, stride=stride, padding=0, bias=False
        )
        self.bn = nn.BatchNorm2d(nOut)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class ResNetBasicblock(nn.Module):
    expansion = 1
    """
    RexNet basicblock (https://github.com/facebook/fb.resnet.torch/blob/master/models/resnet.lua)
    """

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResNetBasicblock, self).__init__()

        self.conv_a = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn_a = nn.BatchNorm2d(planes)

        self.conv_b = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn_b = nn.BatchNorm2d(planes)

        self.downsample = downsample

    def forward(self, x):
        residual = x

        basicblock = self.conv_a(x)
        basicblock = self.bn_a(basicblock)
        basicblock = F.relu(basicblock, inplace=True)

        basicblock = self.conv_b(basicblock)
        basicblock = self.bn_b(basicblock)

        if self.downsample is not None:
            residual = self.downsample(x)

        return F.relu(residual + basicblock, inplace=True)


class CifarResNet(nn.Module):
    """
    ResNet optimized for the Cifar dataset, as specified in
    https://arxiv.org/abs/1512.03385.pdf
    """

    def __init__(self, block, depth, num_classes):
        """Constructor
        Args:
            depth: number of layers.
            num_classes: number of classes
            base_width: base width
        """
        super(CifarResNet, self).__init__()

        # Model type specifies number of layers for CIFAR-10 and CIFAR-100 model
        assert (depth - 2) % 6 == 0, "depth should be one of 20, 32, 44, 56, 110"
        layer_blocks = (depth - 2) // 6
        # print ('CifarResNet : Depth : {} , Layers for each block : {}'.format(depth, layer_blocks))

        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        self.inplanes = 16
        self.layer1 = self._make_layer(block, 16, layer_blocks, stride=1)
        self.layer2 = self._make_layer(block, 32, layer_blocks, stride=2)
        self.layer3 = self._make_layer(block, 64, layer_blocks, stride=2)
        self.relu = nn.ReLU(inplace=True)
        # self.avgpool = nn.AvgPool2d(8)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2.0 / n))
                # m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = DownsampleA(self.inplanes, planes * block.expansion, stride)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def resnet20(num_classes=10):
    """Constructs a ResNet-20 model for CIFAR-10 (by default)
    Args:
        num_classes (uint): number of classes
    """
    model = CifarResNet(ResNetBasicblock, 20, num_classes)
    return model

In [4]:
base_model = resnet20(num_classes=10).cuda()

## Prepare CIFAR10

In [5]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Prepare the CIFAR-100 for training
batch_size = 128
num_workers = 4


cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std),
    ]
)
test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std),
    ]
)

train_dataset = datasets.CIFAR10(
    root="data", train=True, download=True, transform=train_transform
)
test_dataset = datasets.CIFAR10(
    root="data", train=False, download=True, transform=test_transform
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=True,
    worker_init_fn=WorkerInitializer(manualSeed).worker_init_fn,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False,
    worker_init_fn=WorkerInitializer(manualSeed).worker_init_fn,
)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Files already downloaded and verified
Files already downloaded and verified


## Training

In [6]:
max_epoch = 200

optim_setting = {
    "name": "SGD",
    "args": {
        "lr": 0.1,
        "momentum": 0.9,
        "weight_decay": 5e-4,
        "nesterov": True,
    },
}
scheduler_setting = {
    "name": "CosineAnnealingLR",
    "args": {"T_max": max_epoch, "eta_min": 0.0},
}

In [7]:
import os

import torch


def accuracy(
    output: torch.Tensor, target: torch.Tensor, topk: tuple[int,] = (1,)
) -> list[torch.Tensor]:
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(100 * correct_k / batch_size)
    return res


class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val: float = 0.0
        self.avg: float = 0.0
        self.sum: float = 0.0
        self.count: int = 0

    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint(model, save_dir, epoch, is_best=False):
    state = {
        "epoch": epoch,
        "arch": model.__class__.__name__,
        "model_pram": model.state_dict(),
    }
    if is_best:
        path = os.path.join(save_dir, "best_checkpoint.pkl")
    else:
        path = os.path.join(save_dir, "checkpoint_epoch_%d.pkl" % epoch)
    torch.save(state, path, pickle_protocol=4)


def load_checkpoint(model, save_dir, epoch=1, is_best=False):
    if is_best:
        path = os.path.join(save_dir, "best_checkpoint.pkl")
    else:
        path = os.path.join(save_dir, "checkpoint_epoch_%d.pkl" % epoch)
    state = torch.load(path, map_location="cpu")
    model.cpu()
    model.load_state_dict(state["model_pram"])
    model.cuda()


In [8]:
optimizer = getattr(torch.optim, optim_setting["name"])(
    base_model.parameters(), **optim_setting["args"]
)
scheduler = getattr(torch.optim.lr_scheduler, scheduler_setting["name"])(
    optimizer, **scheduler_setting["args"]
)
criterion = nn.CrossEntropyLoss().cuda()
scaler = torch.cuda.amp.GradScaler()
loss_meter=AverageMeter()
top1_meter=AverageMeter()

In [9]:
import time

save_dir = "checkpoint/resnet20"
os.makedirs(save_dir, exist_ok=True)

best_top1 = 0
for epoch in range(1, max_epoch + 1):
    print(f"epoch {epoch}")
    start_time = time.time()
    
    base_model.train()
    for image, label in train_dataloader:
        image = image.cuda()
        label = label.cuda()
        
        base_model.zero_grad()
        optimizer.zero_grad()
        criterion.zero_grad()
        with torch.cuda.amp.autocast():
            y = base_model(image)
            loss = criterion(y, label)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        [top1] = accuracy(y, label, topk=(1,))
        loss_meter.update(loss.item(), label.size(0))
        top1_meter.update(top1.item(), label.size(0))
    scheduler.step()
    print(
        "loss :train={0:.3f}   top1 :train={1:.3f}".format(
            loss_meter.avg, top1_meter.avg
        )
    )
    loss_meter.reset()
    top1_meter.reset()

    base_model.eval()
    for image, label in test_dataloader:
        image = image.cuda()
        label = label.cuda()
        
        with torch.cuda.amp.autocast():
            with torch.no_grad():
                y = base_model(image)
        [top1] = accuracy(y, label, topk=(1,))
        top1_meter.update(top1.item(), label.size(0))
    print(
        "top1 :test={0:.3f}".format(
            top1_meter.avg
        )
    )
    if best_top1 <= top1_meter.avg:
        save_checkpoint(base_model, save_dir, epoch, is_best=True)
        best_top1 = top1_meter.avg
    top1_meter.reset()
    
    elapsed_time = time.time() - start_time
    print("  elapsed_time:{0:.3f}[sec]".format(elapsed_time))

load_checkpoint(base_model, save_dir, is_best=True)

epoch 1
loss :train=1.539   top1 :train=44.159
top1 :test=51.720
  elapsed_time:15.603[sec]
epoch 2
loss :train=1.035   top1 :train=62.997
top1 :test=61.740
  elapsed_time:14.840[sec]
epoch 3
loss :train=0.829   top1 :train=70.907
top1 :test=69.200
  elapsed_time:14.909[sec]
epoch 4
loss :train=0.726   top1 :train=74.752
top1 :test=72.840
  elapsed_time:15.197[sec]
epoch 5
loss :train=0.674   top1 :train=76.567
top1 :test=71.860
  elapsed_time:14.749[sec]
epoch 6
loss :train=0.631   top1 :train=78.265
top1 :test=75.100
  elapsed_time:14.577[sec]
epoch 7
loss :train=0.608   top1 :train=79.044
top1 :test=72.030
  elapsed_time:14.919[sec]
epoch 8
loss :train=0.588   top1 :train=79.665
top1 :test=74.210
  elapsed_time:14.665[sec]
epoch 9
loss :train=0.565   top1 :train=80.533
top1 :test=71.540
  elapsed_time:14.604[sec]
epoch 10
loss :train=0.557   top1 :train=80.731
top1 :test=72.820
  elapsed_time:14.750[sec]
epoch 11
loss :train=0.542   top1 :train=81.342
top1 :test=71.130
  elapsed_tim

## Create mlpackage

In [10]:
class TorchClassificationModel(nn.Module):
    def __init__(self):
        super(TorchClassificationModel, self).__init__()
        self.names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
        self.layers = nn.Sequential(
            base_model,
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.layers(x)
model = TorchClassificationModel()

In [11]:
model.cpu()
model.eval()
image, label = next(iter(test_dataloader))
output = model(image)
accuracy(output, label)

[tensor(96.8750)]

In [12]:
example_input = torch.rand(1, 3, 32, 32) 
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

In [15]:
import coremltools as ct

scale = 1/((sum(cifar10_std)/3)*255.0)
bias = [- cifar10_mean[0]/cifar10_std[0] , - cifar10_mean[1]/cifar10_std[1], - cifar10_mean[2]/cifar10_std[2]]

classifier_config = ct.ClassifierConfig(model.names)
image_input = ct.ImageType(name="image",
                           shape=example_input.shape,
                           scale=scale, bias=bias)

mlmodel = ct.convert(
    traced_model,
    classifier_config=classifier_config,
    inputs=[image_input]
)
mlmodel.save("ResNet20CIFAR10")

When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_targer' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://coremltools.readme.io/docs/unified-conversion-api#target-conversion-formats
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 329.95 passes/s]
